In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import sys
import pandas as pd
import re

In [ ]:
from keras.preprocessing.sequence import utils,pad_sequences
from keras.layers import Embedding,Conv2D,Flatten,BatchNormalization,LSTM,Dropout
from keras.models import Sequential,Input,Model
from keras.preprocessing.text import Tokenizer
import tensorflow as tf

In [ ]:
def read_data(which_set):
    #with open(os.path.join(path_to_project,dataset)+"\\"+which_set) as file:
    train=pd.read_csv(which_set)
    count=0
    for k in train.keys():
        count+=1
    print('number of keys',count)
    return train

In [ ]:
print(os.getcwd())
#os.chdir('../input')
#print(os.getcwd())

In [ ]:
training_data=read_data('../input/train.csv')
test_data=read_data('../input/test.csv')
print("sample data points",training_data.head(10))
print("data keys",training_data.keys())
print("length of each column in training data:",training_data['qid'].shape)
print("length of each column in training data:",test_data['qid'].shape)

In [ ]:
num_data=training_data['qid'].shape[0]
training_data['question_text'][0]
all_sents=training_data['question_text'].tolist()
all_sents_test=test_data['question_text'].tolist()
print(len(all_sents))
print(all_sents[0])

In [ ]:
!pip install nltk
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk import pos_tag
from nltk import NaiveBayesClassifier
from nltk import FreqDist
from nltk.corpus import stopwords
#nltk.download('stopwords')
global stops
stops=stopwords.words('english')

In [ ]:
print("stopwords(english): ",stops)

In [ ]:
def removestops(text_string):
    no_stops=[]
    prepare_word_dict=[]
    for s in text_string:
        words=s.split(' ')
        no_stop_sents=[]
        for w in words:
            if not w in stops:
                w=w.lower()
                w1=re.sub(pattern='\W+',string=w,repl='')
                #if w1 not in prepare_word_dict:
                prepare_word_dict.append(w1)
                no_stop_sents.append(w)
        no_stops.append(no_stop_sents)
    print(len(prepare_word_dict))
    print(prepare_word_dict[0:10])
    return no_stops,prepare_word_dict
no_stops,prepare_word_dict=removestops(all_sents)
no_stops_test,prepare_test_dict=removestops(all_sents_test)

In [ ]:
print(no_stops[0])

In [ ]:
#no_stops_str=[]
#for sent in no_stops:
#    no_stops_str.append(' '.join(sent))
#print(no_stops_str[0])

In [ ]:
#no_specials=[]
#for st in no_stops_str:
#    no_specials.append(re.sub(pattern='\W+',string=st,repl=' '))



def clean_text(s1):
    s1_str=[]
    for sent in s1:
        s1_str.append(' '.join(sent))
    out_st=[re.sub(pattern='\W+',string=st,repl=' ') for st in s1_str]
    return out_st
no_specials=clean_text(no_stops)
no_test_specials=clean_text(no_stops_test)
print('sentence before: ',all_sents[1],'\nsentence after: ',no_specials[1])

Find out Word embeddings using Word2vec/GloVe

In [ ]:
#get vocabulary collection
prepare_word_dict=list(set(prepare_word_dict))
prepare_test_dict=list(set(prepare_test_dict))
'dofs' in prepare_word_dict
print("number of unique words in train and test data: ",len(prepare_word_dict),' ',len(prepare_test_dict))

In [ ]:
#Prepare training data for word2vec
#(center word,context)--> learn one-hot repr of context word(input) from one-hpt repr of center word(target) 
#from keras.preprocessing.text import Tokenizer,one_hot
#onehot_vec=np.zeros(len(prepare_word_dict))
#word2ind={}
#ind2word={}
#for i,word in enumerate(prepare_word_dict):
#    if word!='':
#        word2ind[word]=i
#        ind2word[i]=word
#use_to_train=[]
#for st in no_specials:
#    lst=st.split(' ')
#    if '' in lst:
#        lst.remove('')
#    use_to_train.append(lst)
#print(use_to_train[1:3])
#print(len(use_to_train))


def prep_data_for_word_embeds(s1_dict,s2_no_specs):
    onehot_vec=np.zeros(len(s1_dict))
    word2ind={}
    ind2word={}
    for i,word in enumerate(s1_dict):
        if word!='':
            word2ind[word]=i
            ind2word[i]=word
    out_data=[]
    for st in s2_no_specs:
        lst=st.split(' ')
        if '' in lst:
            lst.remove('')
        out_data.append(lst)
    
    return out_data,word2ind,ind2word

use_to_train,word2ind,ind2word=prep_data_for_word_embeds(prepare_word_dict,no_specials) 
use_to_test,_,_=prep_data_for_word_embeds(prepare_test_dict,no_test_specials)

In [ ]:
def get_input_data(in_data):
    X=[' '.join(ele) for ele in in_data]
    return X
train_X=get_input_data(use_to_train)
test_X=get_input_data(use_to_test)
print("training data: ",train_X[:5])
print("test data: ", test_X[:5])
#emb_model=Word2Vec(sentences=use_to_train,min_count=1,window=window_size,size=50,workers=1)

In [ ]:
#emb_model.train(sentences=use_to_train,epochs=5,total_examples=len(use_to_train))

In [ ]:
#emb_model.most_similar(positive=['xbox', 'football'], negative=['game'], topn=1)
for ele in use_to_train:
    for i,w in enumerate(ele):
        if w in word2ind.keys():
            ele[i]=int(word2ind[w])
use_to_train[:10]

In [ ]:
lengths_train=[]
lengths_test=[]
for ele in use_to_train:
    lengths_train.append(len(ele))
for ele in use_to_test:
    lengths_test.append(len(ele))
maxlen_train=max(lengths_train)
maxlen_test=max(lengths_test)
maxlen=max(maxlen_train,maxlen_test)
maxfeatures=40000
#padded=pad_sequences(sequences=use_to_train,maxlen=None,padding='post')
tokenizer = Tokenizer(num_words=maxfeatures)
tokenizer.fit_on_texts(list(train_X)+list(test_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X=tokenizer.texts_to_sequences(test_X)
train_X = pad_sequences(train_X, maxlen=maxlen)
test_X=pad_sequences(test_X,maxlen=maxlen)
#for ele in use_to_train:
#    for i in range(len(ele))

In [ ]:
print("input train shape: ",train_X.shape)
print("input test shape: ",test_X.shape)

In [ ]:
train_y=training_data['target']
from sklearn.model_selection import train_test_split
train1_X,val_X,train1_y,val_y=train_test_split(train_X,train_y,test_size=0.1,random_state=1)

In [ ]:
from keras.layers import Conv1D,MaxPool2D,LSTM,Input,MaxPool1D,Flatten,Dense,Dropout
from keras.models import Sequential
from keras.utils import multi_gpu_model

In [ ]:

dims=50
model=Sequential()
inputs=Input(shape=(maxlen,))
model.add(Embedding(maxfeatures,dims))
model.add(Conv1D(kernel_size=11,strides=2,filters=64,padding='same',use_bias=True,activation='relu'))
model.add(MaxPool1D(padding='same',pool_size=3,strides=2))
model.add(Conv1D(kernel_size=6,strides=2,filters=128,activation='relu'))
model.add(MaxPool1D(padding='same',pool_size=2,strides=2))
model.add(Conv1D(kernel_size=3,strides=1,filters=128,padding='same',use_bias=True,activation='relu'))
model.add(MaxPool1D(padding='same',pool_size=2,strides=2))
model.add(LSTM(units=64,dropout=0.9,return_sequences=False))
model.add(Dense(activation='relu',units=64))
model.add(Dropout(rate=0.1))
model.add(Dense(activation='relu',units=10))
model.add(Dropout(rate=0.1))
model.add(Dense(activation='sigmoid',units=1))

In [ ]:
model.summary()

In [ ]:


model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.fit(batch_size=256,epochs=10,x=train1_X,y=train1_y,shuffle=True,validation_data=(val_X,val_y))

In [ ]:
def savemodel(model):
    model_json = model.to_json()
    with open(os.path.join('..',"model.json"), "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    #model.save_weights("model.h5")
    print('=====model has been saved in JSON format======')
    print('=====model has been saved in  format======')
print("model has been saved in: ",os.getcwd())

In [ ]:
#savemodel(model)

In [ ]:
predictions=model.predict(test_X)

In [ ]:
predictions[0:100]

In [ ]:
test_probs=np.squeeze(predictions)
pred_classes=np.zeros(np.shape(test_probs))
for i in range(test_probs.shape[0]):
    if test_probs[i]<0.5:
        pred_classes[i]=0
    else:
        pred_classes[i]=1
print(pred_classes)

In [ ]:
len(pred_classes[pred_classes==1])

In [ ]:
df=pd.DataFrame(data={'qid':test_data['qid'],'prediction':pred_classes})

In [ ]:
df.to_csv(header=True,sep=',',index=False,path_or_buf='input/submission.csv')

In [ ]:
os.listdir('..')